In [1]:
import pyspark.sql.functions as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
        .appName("YouTube_Model_USA")
        # allocate 8 GB to the driver JVM:
        .config("spark.driver.memory", "8g")
        # when running in local mode this also inflates the executor memory:
        .config("spark.executor.memory", "8g")
        # limit the size of results Spark will try to pull back to Python
        .config("spark.driver.maxResultSize", "2g")
        .getOrCreate()
)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 03:54:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# to hide long warning messages in console output
spark.sparkContext.setLogLevel("ERROR")

In [4]:
# Load the data
import os
current_path = os.getcwd()
full_path = f'file://{current_path}/US_youtube_trending_data_clean.csv'
df = spark.read.csv(full_path, inferSchema = True, header = True)
df = df.drop("_c15")


In [5]:
df = df.filter(F.col("view_count") != 0)
df = df.filter(F.col("comments_disabled") != 'true')

In [6]:
# adding title statistics and seeing statistics 
# Create a column for the length of each title (number of characters)
df = df.withColumn("title_length", F.length("title"))

# Create a column for the word count (splitting on whitespace)
df = df.withColumn("word_count", F.size(F.split("title", " ")))

In [7]:
df = df.withColumn("publish_year", F.year("publishedAt")).withColumn("publish_month", F.month("publishedAt")).withColumn("publish_dayofweek", F.dayofweek("publishedAt")).withColumn("publish_hour", F.hour("publishedAt"))

In [8]:
#Catagorical columns
# mapping categoryid to its value through joining json file
categories_df = spark.read.option("multiLine", "true").json("US_category_id.json")
categories_df = categories_df.select(F.explode("items").alias("item"))
categories_df = categories_df.select(
    F.col("item.id").alias("categoryId"),
    F.col("item.snippet.title").alias("categoryTitle")
)

# Cast categoryId to integer so it matches your main DataFrame
categories_df = categories_df.withColumn("categoryId", F.col("categoryId").cast("integer"))

In [9]:
df_with_categories = df.join(categories_df, on="categoryId", how="left")

In [10]:
df = df.withColumn("dislikes_ratio", F.round(F.col("dislikes") / F.col("view_count"), 10))

In [11]:
df = df.withColumn("comments_ratio", F.col("comment_count")/F.col("view_count"))

In [12]:
df = df.withColumn("likes_ratio", F.round(F.col("likes") / F.col("view_count"), 2))

In [13]:
df_tags_split = df.withColumn("tag_array", F.split(F.col("tags"), r"\|"))

In [14]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import HashingTF, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# Define UDF to lowercase each element in the tag array
def lower_array(tags):
    if tags is None:
        return []
    return [tag.lower() for tag in tags]

lower_udf = F.udf(lower_array, ArrayType(StringType()))

# Create a new DataFrame so that original data is not overwritten.
# Lowercase the tag array column and store it in a new column 'tag_array_lower'
df_ml = df_tags_split.withColumn("tag_array_lower", lower_udf(F.col("tag_array")))


In [15]:
df_ml = df_ml.withColumn("channelTitle", F.lower(F.col("channelTitle")))

In [16]:
# Cast the selected numerical columns to string so we can treat them as categorical
df_ml = df_ml.withColumn("categoryID_str", F.col("categoryId").cast("string")) \
             .withColumn("title_length_str", F.col("title_length").cast("string")) \
             .withColumn("word_count_str", F.col("word_count").cast("string")) \
             .withColumn("publish_month_str", F.col("publish_month").cast("string")) \
             .withColumn("publish_dayofweek_str", F.col("publish_dayofweek").cast("string")) \
             .withColumn("publish_hour_str", F.col("publish_hour").cast("string"))

# Define the list of our categorical columns (as strings)
cat_cols = ["categoryID_str", "title_length_str", "word_count_str", 
            "publish_month_str", "publish_dayofweek_str", "publish_hour_str", "channelTitle"]


In [15]:
# getting unique tags to use as buckets in hashingtf
unique_tag_count = df_ml.select(F.explode(F.col("tag_array_lower"))).distinct().count()
print("Unique tags:", unique_tag_count)


Unique tags: 225644


In [17]:
# For each categorical column, create a StringIndexer (with handleInvalid="keep" to avoid errors)
indexers = [StringIndexer(inputCol=col_name, outputCol=col_name + "_idx", handleInvalid="keep")
            for col_name in cat_cols]

# Next, use a OneHotEncoder to convert the indexed categories to one-hot encoded vectors
encoders = [OneHotEncoder(inputCol=col_name + "_idx", outputCol=col_name + "_ohe")
            for col_name in cat_cols]

# Create the HashingTF stage to convert the preprocessed tag array into a fixed-length feature vector.
hashing_tf = HashingTF(inputCol="tag_array_lower", outputCol="hashing_tf_features", numFeatures=262144)

# Assemble all features into a single vector. We use the HashingTF features
# as well as all the one-hot encoded categorical columns.
assembler_inputs = ["hashing_tf_features"] + [col_name + "_ohe" for col_name in cat_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Create the pipeline by combining all stages
pipeline_stages = []
pipeline_stages += indexers
pipeline_stages += encoders
pipeline_stages.append(hashing_tf)
pipeline_stages.append(assembler)

pipeline = Pipeline(stages=pipeline_stages)

In [18]:
# Rename the target variable 'comments_ratio' to 'label' for cr modleing
df_ml_cr = df_ml.withColumnRenamed("comments_ratio", "label")

# Fit the pipeline model on the data and transform the DataFrame.
pipeline_model = pipeline.fit(df_ml_cr)
df_ml_cr_final = pipeline_model.transform(df_ml_cr)

In [19]:
# linear rgreression predict comments ratio

train, test = df_ml_cr_final.randomSplit([0.7, 0.3], seed=101)

# Build a Linear Regression model to predict the label (comments_ratio)
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model_cr = lr.fit(train)

# Evaluate the model on the test set
test_results = lr_model_cr.evaluate(test)
print("Building model for target: comments_ratio")
print("Test RMSE:", test_results.rootMeanSquaredError)
print("Test R2:", test_results.r2)

[Stage 130:==========================================>              (3 + 1) / 4]

Building model for target: comments_ratio
Test RMSE: 0.0022260137843659288
Test R2: 0.7727452169518236


In [20]:
# likes ratio for modeling
df_ml_lr = df_ml.withColumnRenamed("likes_ratio", "label")

# Fit the pipeline model on the data and transform the DataFrame.
pipeline_model = pipeline.fit(df_ml_lr)
df_ml_lr_final = pipeline_model.transform(df_ml_lr)

# linear rgreression predict likes ratio

train, test = df_ml_lr_final.randomSplit([0.7, 0.3], seed=101)

from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model_lr = lr.fit(train)

# Evaluate the model on the test set
test_results = lr_model_lr.evaluate(test)
print("Building model for target: likes_ratio")
print("Test RMSE:", test_results.rootMeanSquaredError)
print("Test R2:", test_results.r2)

[Stage 260:============================>                            (2 + 2) / 4]

Building model for target: likes_ratio
Test RMSE: 0.014367431738537398
Test R2: 0.8455419364540924


### Count Vectorizer

In [21]:
# Build the new pipeline: CountVectorizer + OneHot + Assembler
from pyspark.ml.feature import CountVectorizer, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# CountVectorizer on tags  
cv = CountVectorizer(inputCol="tag_array_lower", outputCol="tagVec",
                     vocabSize=225644, minDF=0) # keep every tag

# index & OHE for our categorical strs
indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep") for c in cat_cols]
encoders  = [OneHotEncoder(inputCol=c+"_idx", outputCol=c+"_ohe")          for c in cat_cols]

# assemble
assembler_inputs = ["tagVec"] + [c+"_ohe" for c in cat_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

pipeline = Pipeline(stages=[cv] + indexers + encoders + [assembler])


# Model for comments_ratio
from pyspark.ml.regression import LinearRegression

df_ml_cr    = df_ml.withColumnRenamed("comments_ratio", "label")
pipe_model  = pipeline.fit(df_ml_cr)
df_cr_final = pipe_model.transform(df_ml_cr)

train_cr, test_cr = df_cr_final.randomSplit([0.7,0.3], seed=101)
lr_cr = LinearRegression(featuresCol="features", labelCol="label")
lr_model_cr = lr_cr.fit(train_cr)
res_cr      = lr_model_cr.evaluate(test_cr)

print("Comments_ratio → CountVec:  RMSE =", res_cr.rootMeanSquaredError,
      ", R2 =", res_cr.r2)

# Model for likes_ratio
df_ml_lr    = df_ml.withColumnRenamed("likes_ratio", "label")
df_lr_final = pipe_model.transform(df_ml_lr)

train_lr, test_lr = df_lr_final.randomSplit([0.7,0.3], seed=101)
lr_lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model_lr = lr_lr.fit(train_lr)
res_lr      = lr_model_lr.evaluate(test_lr)

print("Likes_ratio    → CountVec:  RMSE =", res_lr.rootMeanSquaredError,
      ", R2 =", res_lr.r2)

Comments_ratio → CountVec:  RMSE = 0.0022150931130384644 , R2 = 0.7749695401810701


[Stage 504:==========================================>              (3 + 1) / 4]

Likes_ratio    → CountVec:  RMSE = 0.01428161534611369 , R2 = 0.847381575995003
